In [4]:
# !pip install textblob
# !pip install vaderSentiment

In [1]:
from textblob import TextBlob
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

dataset = pd.read_csv("finalData.csv")
print(dataset.count())
# dataset = dataset[['description']]

Unnamed: 0     32120
index          32120
description    32120
category       17120
civic_issue    32120
dtype: int64


In [2]:
def sentiment_analysis(dataset):
    def getSubjectivity(text):
        return TextBlob(text).sentiment.subjectivity
# Create a function to get the polarity
    def getPolarity(text):
        return TextBlob(text).sentiment.polarity

    def getAnalysis(score):
        if score < 0.3:
            return 'Complaint'
        elif score > 0.3 and score < 0.4:
            return 'Neutral Feedback'
        else:
            return 'Compliment'

    dataset['Polarity'] = dataset.description.apply(getPolarity)
    dataset['Sentiment'] = dataset['Polarity'].apply(getAnalysis )
    return dataset

df = sentiment_analysis(dataset)
#  if score < 0:
#   return ‘Negative’
#  elif score == 0:
#   return ‘Neutral’
#  else:
#   return ‘Positive’

In [3]:
df.head(10)

,Unnamed: 0,index,description,category,civic_issue,Polarity,Sentiment
0,0,1,Garbage behind the temple,Waste/Garbage,1,-0.400000,Complaint
1,1,2,Air pollution,Air,1,0.000000,Complaint
2,2,3,Air pollution in hebbal,Air,1,0.000000,Complaint
3,3,4,Garbage is dumped near BES,Waste/Garbage,1,0.100000,Complaint
4,4,5,Leaf and garbage burning on Shakthi Ganapathi ...,Air,1,-0.050000,Complaint
5,5,6,There is a lot of garbage dump in and around t...,Waste/Garbage,1,-0.250000,Complaint
6,6,7,Replace broken streetlight. Complaint #5 about...,Streetlights,1,-0.233333,Complaint
7,7,8,Red Matiz parked on the footpath for more than...,Traffic/Parking,1,0.150000,Complaint
8,8,9,Cars have been using the footpath as a parking...,Traffic/Parking,1,-0.125000,Complaint
9,9,10,Congestion everyday and at all times due to im...,Traffic/Parking,1,-0.162500,Complaint


### Using VADERSentiment - Valence Aware Dictionary and sEntiment Reasoner

In [6]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

sentimentAnalyser = SentimentIntensityAnalyzer()

def calculate_sentiment_analyser(text):    
    return sentimentAnalyser.polarity_scores(text)

df['Sentiment'] = df['description'].apply(calculate_sentiment_analyser)
df.head()

,Unnamed: 0,index,description,category,civic_issue,Polarity,Sentiment
0,0,1,Garbage behind the temple,Waste/Garbage,1,-0.40,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
1,1,2,Air pollution,Air,1,0.00,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
2,2,3,Air pollution in hebbal,Air,1,0.00,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
3,3,4,Garbage is dumped near BES,Waste/Garbage,1,0.10,"{'neg': 0.403, 'neu': 0.597, 'pos': 0.0, 'comp..."
4,4,5,Leaf and garbage burning on Shakthi Ganapathi ...,Air,1,-0.05,"{'neg': 0.134, 'neu': 0.729, 'pos': 0.137, 'co..."


In [16]:
s = pd.DataFrame(index = range(0,len(df)),columns= ['compound_score','compound_score_sentiment'])

for i in range(0,len(df)): 
  s['compound_score'][i] = df['Sentiment'][i]['compound']
  
  if (df['Sentiment'][i]['compound'] <= 0.05):
    s['compound_score_sentiment'][i] = 'Negative'    
  if (df['Sentiment'][i]['compound'] >= 0.25):
    s['compound_score_sentiment'][i] = 'Positive'
  if ((df['Sentiment'][i]['compound'] >= 0.05) & (df['Sentiment'][i]['compound'] <= 0.25)):
    s['compound_score_sentiment'][i] = 'Neutral'
    
df['sentiment_score'] = s['compound_score']
df['sentiment'] = s['compound_score_sentiment']
df = df.drop(columns=['compound_score','compound_score_sentiment'])
df.head()

,Unnamed: 0,index,description,category,civic_issue,Polarity,Sentiment,sentiment_score,sentiment
0,0,1,Garbage behind the temple,Waste/Garbage,1,-0.40,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0,Negative
1,1,2,Air pollution,Air,1,0.00,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0,Negative
2,2,3,Air pollution in hebbal,Air,1,0.00,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0,Negative
3,3,4,Garbage is dumped near BES,Waste/Garbage,1,0.10,"{'neg': 0.403, 'neu': 0.597, 'pos': 0.0, 'comp...",-0.4019,Negative
4,4,5,Leaf and garbage burning on Shakthi Ganapathi ...,Air,1,-0.05,"{'neg': 0.134, 'neu': 0.729, 'pos': 0.137, 'co...",0.015,Negative


In [13]:
df.head(20)
df.to_csv("sentiments.csv")

### Using K-Means Clustering on the Polarity attribute